In [4]:
import variableImageSize as vis
import os
from glob import glob
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch

In [5]:
# Path to dataset
root_dir = "./data/MNIST/mnist-varres/test/"

# Load images grouped by resolution
tensor_32, tensor_48, tensor_64 = vis.load_images_by_resolution(root_dir)

# Print the size of each group
print(f"32x32: Images {tensor_32[0].shape}, Labels {tensor_32[1].shape}")
print(f"48x48: Images {tensor_48[0].shape}, Labels {tensor_48[1].shape}")
print(f"64x64: Images {tensor_64[0].shape}, Labels {tensor_64[1].shape}")

32x32: Images torch.Size([3269, 1, 32, 32]), Labels torch.Size([3269])
48x48: Images torch.Size([3381, 1, 48, 48]), Labels torch.Size([3381])
64x64: Images torch.Size([3350, 1, 64, 64]), Labels torch.Size([3350])


In [6]:
model = vis.VariableInputNetwork(num_classes=10, N=64, pooling_type='max')
print(model)

# Test with variable input sizes
input_32 = torch.randn(8, 1, 32, 32)  # Batch of 8 images, resolution 32x32
input_48 = torch.randn(8, 1, 48, 48)  # Batch of 8 images, resolution 48x48
input_64 = torch.randn(8, 1, 64, 64)  # Batch of 8 images, resolution 64x64

output_32 = model(input_32)  # Should output (8, 10)
output_48 = model(input_48)  # Should output (8, 10)
output_64 = model(input_64)  # Should output (8, 10)

print(f"Output shape for 32x32 input: {output_32.shape}")
print(f"Output shape for 48x48 input: {output_48.shape}")
print(f"Output shape for 64x64 input: {output_64.shape}")

VariableInputNetwork(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_pool): AdaptiveMaxPool2d(output_size=(1, 1))
  (fc): Linear(in_features=64, out_features=10, bias=True)
)
Output shape for 32x32 input: torch.Size([8, 10])
Output shape for 48x48 input: torch.Size([8, 10])
Output shape for 64x64 input: torch.Size([8, 10])


In [8]:
print(vis.count_parameters(model))

AttributeError: module 'variableImageSize' has no attribute 'count_parameters'

In [7]:
#TODO TRAINING LOOP FOR THIS